In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn import manifold
from scipy import stats
from itertools import cycle
import sys
import utilities as ut
from Network import Net
import Network as nn


results = []
num_classes = 2
sc = '_RRd'
learning_rate = 0.005
samples = 3000

for epsilon in [0.2]:
    for batch_size in [256]:
        for hidden_size in [100]:
            for EPS1 in [0.025]:
                for n in [100, 200, 400]:
                    for aux_loss_activated in [True]:
                        for opt in [2]:
                            for t in range(30):
                                train_dataset, test_dataset = ut.load_files(dataset=1, subclass=sc)
                                input_size = train_dataset.shape[1]-1
                                print('before cleaning outliers')
                                print(train_dataset.shape)
                                print(test_dataset.shape)

                                train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)                                
                                print('after cleaning outliers')
                                print(train_dataset.shape)
                                print(test_dataset.shape)

                                if n < 50000:
                                    train_dataset = ut.down_sampling(train_dataset)
                                    train_dataset = train_dataset.sample(n)
                                    print(train_dataset)
                                else: 
                                    trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                    print('shape: ', trainig_dataset_a.shape[0])
                                    n2 = n - trainig_dataset_a.shape[0]
                                    print('clase no RR Lrae', n2)
                                    trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                    train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
                                

                                train_dataset = ut.sort_columns(train_dataset)
                                test_dataset = ut.sort_columns(test_dataset)
                                #... normalize ...
                                
                                #print(train_dataset.columns)
                                test_dataset_pred = test_dataset.copy()
                                train_dataset_pred = train_dataset.copy()
                                try:
                                    data_prior = ut.generate_samples(samples, train_dataset, epsilon,  option = opt)

                                    if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
                                        samples_prior = samples 
                                    else: 
                                        samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0]
                                    
                                    data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])


                                    train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)
                                    print(data_prior)
                                    train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

                                    train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2)
                                    print(train_dataset_prior.columns)


                                    _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                                    _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                                    _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                                    _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                                    _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                                    _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                                    _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                                    net = Net(input_size, hidden_size, hidden_size, num_classes)
                                    net.cuda()

                                    hist_val, hist_train = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                                    EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated)

                                    acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                                    acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                                    
                                    roc_train = nn.get_roc_curve(net, train_loader, input_size)
                                    roc_test = nn.get_roc_curve(net, test_loader, input_size)

                                    results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, roc_train, roc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])
                                    pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test', 
                                                                   'roc_train', 'roc_test', 'epsilon', 'batch_size', 'hidden_size',
                                     'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('03-06-2022-sc-1D-period-RRLyrae.csv')
                                except Exception as error:
                                    print(error) 
                                    print(str(epsilon)+"-"+str(batch_size)+"-"+str(hidden_size)+"-"+str(aux_loss_activated)+"-"+str(EPS1))

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/franciscoperez/anacon

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0
downsampling
868
373025
done downsampling
        Amplitude  AndersonDarling  Autocor_length  Beyond1Std    CAR_mean  \
12464      0.2065         0.063228               1    0.360000   27.176965   
145359     0.1450         0.158676               5    0.340000    0.410639   
227314     0.0705         0.000000              18    0.229008   82.144615   
71412      0.2520         0.006453               1    0.376000   28.182056   
337734     0.1725         0.030700               1    0.363333   44.361157   
...           ...              ...             ...         ...         ...   
286520     0.2565         0.637945               1    0.343333  252.356977   
123984     0.0665         0.045629               3    0.296667   16.216161   
345755     0.2720         0.021013               1    0.346667   43.952076   
96076      0.0170         0.168609               2    0.329897  226.027191   
285394     0.025

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


{'fc1.weight': Parameter containing:
tensor([[ 0.0881, -0.0373, -0.1018,  ...,  0.0924, -0.0471,  0.0976],
        [ 0.0185, -0.0602, -0.0193,  ...,  0.0958,  0.0402, -0.0956],
        [ 0.0215,  0.0474,  0.0546,  ...,  0.0689,  0.0216, -0.0371],
        ...,
        [ 0.0513,  0.0602,  0.0887,  ...,  0.0342,  0.1056, -0.0317],
        [ 0.1274, -0.0612, -0.0993,  ...,  0.0989,  0.0810, -0.0385],
        [-0.0598,  0.0592,  0.0642,  ...,  0.0159,  0.1114, -0.1195]],
       device='cuda:0', requires_grad=True), 'fc1.bias': Parameter containing:
tensor([ 0.0819,  0.1168,  0.0092, -0.0977, -0.0334,  0.0168, -0.0607, -0.1084,
         0.0218, -0.0205,  0.0365, -0.0564, -0.0323,  0.1277, -0.0459, -0.0979,
        -0.0837,  0.0019,  0.0536,  0.0964, -0.0195,  0.0190, -0.0107,  0.0416,
        -0.0294, -0.1222, -0.0729, -0.0604, -0.1115, -0.0666, -0.0920,  0.0011,
        -0.1113,  0.0573, -0.0527, -0.1096, -0.0561, -0.0496,  0.0830,  0.0979,
        -0.0366,  0.0092, -0.0460,  0.0747, -0.037

the_last_loss:  0.8529929518699646
running_loss_val:  0.8462038040161133
validating: epoch:  40  loss:  0.8462038040161133
The current loss: 0.8462038040161133
the_last_loss: 0.8529929518699646
trigger times: 0
sum mask2 - L1:  tensor(4894, device='cuda:0')
sum mask2 - L2:  tensor(7470, device='cuda:0')
sum mask2 - L3:  tensor(153, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1106, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2530, device='cuda:0')
sum mask1 - L3 (aux):  tensor(47, device='cuda:0')
training: epoch:  41  loss:  0.6313339471817017 -- aux loss:  0.0
training: epoch:  42  loss:  0.6310478448867798 -- aux loss:  0.0
training: epoch:  43  loss:  0.6307920217514038 -- aux loss:  0.0
training: epoch:  44  loss:  0.6305623054504395 -- aux loss:  0.0
training: epoch:  45  loss:  0.630355715751648 -- aux loss:  0.0
training: epoch:  46  loss:  0.6301692724227905 -- aux loss:  0.0
training: epoch:  47  loss:  0.630000650882721 -- aux loss:  0.0
training: epoch:  48  loss:  0.6

...

training: epoch:  1179  loss:  0.6265435814857483 -- aux loss:  0.0
training: epoch:  1180  loss:  0.6265435218811035 -- aux loss:  0.0
the_last_loss:  0.8831294775009155
running_loss_val:  0.8259838223457336
validating: epoch:  1180  loss:  0.8259838223457336
The current loss: 0.8259838223457336
the_last_loss: 0.8831294775009155
trigger times: 0
sum mask2 - L1:  tensor(4894, device='cuda:0')
sum mask2 - L2:  tensor(7470, device='cuda:0')
sum mask2 - L3:  tensor(153, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1106, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2530, device='cuda:0')
sum mask1 - L3 (aux):  tensor(47, device='cuda:0')
training: epoch:  1181  loss:  0.626543402671814 -- aux loss:  0.0
training: epoch:  1182  loss:  0.6265435218811035 -- aux loss:  0.0
training: epoch:  1183  loss:  0.626543402671814 -- aux loss:  0.0
training: epoch:  1184  loss:  0.6265434622764587 -- aux loss:  0.0
training: epoch:  1185  loss:  0.6265434622764587 -- aux loss:  0.0
training: epoch:

training: epoch:  1264  loss:  0.626541018486023 -- aux loss:  0.0
training: epoch:  1265  loss:  0.6265409588813782 -- aux loss:  0.0
training: epoch:  1266  loss:  0.6265409588813782 -- aux loss:  0.0
training: epoch:  1267  loss:  0.6265409588813782 -- aux loss:  0.0
training: epoch:  1268  loss:  0.6265408396720886 -- aux loss:  0.0
training: epoch:  1269  loss:  0.6265408396720886 -- aux loss:  0.0
training: epoch:  1270  loss:  0.6265409588813782 -- aux loss:  0.0
the_last_loss:  0.7969409823417664
running_loss_val:  0.737213134765625
validating: epoch:  1270  loss:  0.737213134765625
The current loss: 0.737213134765625
the_last_loss: 0.7969409823417664
trigger times: 0
sum mask2 - L1:  tensor(4894, device='cuda:0')
sum mask2 - L2:  tensor(7470, device='cuda:0')
sum mask2 - L3:  tensor(153, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1106, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2530, device='cuda:0')
sum mask1 - L3 (aux):  tensor(47, device='cuda:0')
training: epoch:  

the_last_loss:  0.935774028301239
running_loss_val:  0.6177787184715271
validating: epoch:  1350  loss:  0.6177787184715271
The current loss: 0.6177787184715271
the_last_loss: 0.935774028301239
trigger times: 0
sum mask2 - L1:  tensor(4894, device='cuda:0')
sum mask2 - L2:  tensor(7470, device='cuda:0')
sum mask2 - L3:  tensor(153, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1106, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2530, device='cuda:0')
sum mask1 - L3 (aux):  tensor(47, device='cuda:0')
training: epoch:  1351  loss:  0.626538872718811 -- aux loss:  0.0
training: epoch:  1352  loss:  0.6265388131141663 -- aux loss:  0.0
training: epoch:  1353  loss:  0.6265386343002319 -- aux loss:  0.0
training: epoch:  1354  loss:  0.6265386343002319 -- aux loss:  0.0
training: epoch:  1355  loss:  0.6265386343002319 -- aux loss:  0.0
training: epoch:  1356  loss:  0.6265386939048767 -- aux loss:  0.0
training: epoch:  1357  loss:  0.6265386343002319 -- aux loss:  0.0
training: epoch: 

training: epoch:  1439  loss:  0.6265369653701782 -- aux loss:  0.0
training: epoch:  1440  loss:  0.6265368461608887 -- aux loss:  0.0
the_last_loss:  0.7966195344924927
running_loss_val:  0.7074396014213562
validating: epoch:  1440  loss:  0.7074396014213562
The current loss: 0.7074396014213562
the_last_loss: 0.7966195344924927
trigger times: 0
sum mask2 - L1:  tensor(4894, device='cuda:0')
sum mask2 - L2:  tensor(7470, device='cuda:0')
sum mask2 - L3:  tensor(153, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1106, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2530, device='cuda:0')
sum mask1 - L3 (aux):  tensor(47, device='cuda:0')
training: epoch:  1441  loss:  0.6265369057655334 -- aux loss:  0.0
training: epoch:  1442  loss:  0.6265368461608887 -- aux loss:  0.0
training: epoch:  1443  loss:  0.6265368461608887 -- aux loss:  0.0
training: epoch:  1444  loss:  0.6265368461608887 -- aux loss:  0.0
training: epoch:  1445  loss:  0.6265367269515991 -- aux loss:  0.0
training: epoc

KeyboardInterrupt: 